Evolution under Hamiltonian of Heisenberg interaction in an external field, nearest neighbor line

We are choosing a circuit that has a linear connectivity map which is the simplest possible. The implemented circuit uses a lot of single qubit gates to implement spin interaction along the different axes and the goal is to see whether a custom compiler does better at reducing these gates than the default one.

In [10]:
from qiskit import QuantumCircuit

import time
import numpy as np

Define circuit

In [11]:
num_qubits = 5
num_layers = 3
qc=QuantumCircuit(num_qubits)
for i_layer in range(num_layers):
    for i in range(num_qubits-1):    
        j = (i + 1)
    
        qc.rz(np.random.rand(), i)
        qc.rz(np.random.rand(), j)
        
        qc.cx(i, j)
        qc.rz(0.1, j)
        qc.cx(i, j)
    
        qc.h(i)
        qc.h(j)
        qc.cx(i, j)
        qc.rz(0.1, j)
        qc.cx(i, j)
        qc.h(i)
        qc.h(j)
    
        qc.ry(np.pi/2, i)
        qc.ry(np.pi/2, j)
        qc.cx(i, j)
        qc.rz(0.1, j)
        qc.cx(i, j)
        qc.ry(-np.pi/2, i)
        qc.ry(-np.pi/2, j)
# Get gate counts
gate_counts = qc.count_ops()
print(gate_counts)

OrderedDict({'cx': 72, 'rz': 60, 'h': 48, 'ry': 48})


Define backend

In [12]:
from qiskit.providers.fake_provider import GenericBackendV2

backend = GenericBackendV2(num_qubits=num_qubits, basis_gates = ['rz', 'rx', 'ry', 'cx'])

Optionally print out the circuit

In [13]:
# qc.draw()

Compile with default passes

In [14]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
default_pm = generate_preset_pass_manager(backend=backend, optimization_level=3)

t1 = time.time()
default_qc = default_pm.run(qc)
t2 = time.time()
print("Time taken: ", t2-t1)

# Get gate counts
gate_counts = default_qc.count_ops()
print(gate_counts)
print("Number of 2-qubit gates: ", gate_counts.get("cx", 0)) 
print("Number of 1-qubit gates: ", gate_counts.get("rx", 0) + gate_counts.get("rz", 0) + gate_counts.get("ry", 0) + gate_counts.get("h", 0))

Time taken:  0.11327385902404785
OrderedDict({'rz': 52, 'rx': 40, 'cx': 24, 'ry': 13})
Number of 2-qubit gates:  24
Number of 1-qubit gates:  105


In [15]:
# default_qc.draw()

In [16]:
#Test with UCC transpiler

import sys
sys.path.append('../')

from ucc import compile

t1 = time.time()
ucc_qc, gate_counts = compile(qc, mode="ucc", get_gate_counts = True)
t2 = time.time()
print("Time taken: ", t2-t1)

# # Get gate counts
print(gate_counts)
print("Number of 2-qubit gates: ", gate_counts.get("cx", 0))
print("Number of 1-qubit gates: ", gate_counts.get("rz", 0) + gate_counts.get("rx", 0) + gate_counts.get("ry", 0) + gate_counts.get("h", 0))

Time taken:  0.10228610038757324
OrderedDict({'rz': 60, 'ry': 42, 'cx': 24, 'rx': 11})
Number of 2-qubit gates:  24
Number of 1-qubit gates:  113


Create custom compiler

In [17]:
# from qiskit.transpiler import PassManager
# import qiskit.transpiler.passes as passes 


# custom_pm = PassManager()

# custom_pm.append(passes.Optimize1qGatesDecomposition())

# custom_pm.append(passes.Collect2qBlocks())
# custom_pm.append(passes.ConsolidateBlocks())

# # custom_pm.append(passes.Decompose())
# # custom_pm.append(passes.Optimize1qGates())

# custom_qc = custom_pm.run(qc)

# # Get gate counts
# gate_counts = custom_qc.count_ops()
# print(gate_counts)

# # custom_qc.draw()

In [18]:
# # Get gate counts
# gate_counts = custom_qc.count_ops()
# print(gate_counts)
# print("Number of 2-qubit gates: ", gate_counts.get("cz", 0) + gate_counts.get("cx", 0))
# print("Number of 1-qubit gates: ", gate_counts.get("sx", 0) + gate_counts.get("rz", 0) + gate_counts.get("x", 0))
